In [ ]:
!pip install beautifulsoup4
!pip install requests
!pip install yfinance
!pip install datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 8.6 MB/s eta 0:00:00


In [ ]:
import yfinance as yf
import requests
import bs4 as bs
import datetime as dt
import pandas as pd

In [ ]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})

tickers = []

for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[0].text
    tickers.append(ticker)

In [ ]:
tickers = [s.replace('\n', '') for s in tickers]
tickers[0]

'MMM'

In [ ]:
data = yf.download(tickers, start="2024-01-01", end="2024-03-06", interval = '1mo')
data.head()

[*********************100%%**********************]  503 of 503 completed
ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1mo 2024-01-01 -> 2024-03-06)')
ERROR:yfinance:['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


Price        Adj Close                                                         \
Ticker               A    AAL        AAPL        ABBV        ABNB         ABT   
Date                                                                            
2024-01-01  130.100006  14.23  184.164993  162.844131  144.139999  112.606010   
2024-02-01  137.360001  15.68  180.519653  176.050003  157.470001  118.639999   
2024-03-01  146.220001  14.88  169.119995  181.059998  163.869995  119.339996   

Price                                                      ...    Volume  \
Ticker           ACGL         ACN        ADBE         ADI  ...      WYNN   
Date                                                       ...             
2024-01-01  82.430000  362.556091  617.780029  191.457825  ...  40782700   
2024-02-01  87.589996  374.779999  560.280029  190.920364  ...  40076900   
2024-03-01  88.820000  380.829987  543.090027  190.800919  ...   8141000   

Price                                                                    \
Ticker           XEL        XOM      XRAY       XYL       YUM       ZBH   
Date                                                                      
2024-01-01  96934200  413171900  49517100  23309600  32385100  35313000   
2024-02-01  89039600  360899900  76162500  29551500  44333100  35544600   
2024-03-01  56172900   77018000   7583500   3773600   8182200   4460000   

Price                                    
Ticker         ZBRA      ZION       ZTS  
Date                                     
2024-01-01  8332000  54437800  39437100  
2024-02-01  9010100  62019000  47467800  
2024-03-01  1344500  14913000  15018500  

[3 rows x 3018 columns]

In [ ]:
df = data.stack().reset_index().rename(index=str, columns={"level_1": "Ticker"}).sort_values(['Ticker','Date'])
df.reset_index()
df.head()

Price,Date,Ticker,Adj Close,Close,High,Low,Open,Volume
0,2024-01-01,A,130.100006,130.100006,140.589996,126.739998,138.190002,30229800.0
501,2024-02-01,A,137.360001,137.360001,139.880005,128.259995,130.059998,32189900.0
1002,2024-03-01,A,146.220001,146.220001,147.440002,134.669998,137.029999,8680600.0
1,2024-01-01,AAL,14.230000,14.230000,15.840000,12.780000,13.650000,938715300.0
502,2024-02-01,AAL,15.680000,15.680000,15.790000,13.870000,14.350000,567758400.0


In [ ]:
#calculate 3 month returns. will drop month of feb and do math on jan and march
mask = df['Date'] == '2024-02-01'
df = df[~mask]
df = df[['Date','Ticker','Adj Close']]
df.head()

Price,Date,Ticker,Adj Close
0,2024-01-01,A,130.100006
1002,2024-03-01,A,146.220001
1,2024-01-01,AAL,14.230000
1003,2024-03-01,AAL,14.880000
2,2024-01-01,AAPL,184.164993


In [ ]:
df.head(20)

Price,Date,Ticker,Adj Close
0,2024-01-01,A,130.100006
1002,2024-03-01,A,146.220001
1,2024-01-01,AAL,14.230000
1003,2024-03-01,AAL,14.880000
2,2024-01-01,AAPL,184.164993
1004,2024-03-01,AAPL,169.119995
3,2024-01-01,ABBV,162.844131
1005,2024-03-01,ABBV,181.059998
4,2024-01-01,ABNB,144.139999
1006,2024-03-01,ABNB,163.869995


In [ ]:
df.groupby('Date').count()

Price,Ticker,Adj Close
Date,,
2024-01-01,501,501
2024-03-01,501,501


In [ ]:
df['diffs'] = df.groupby(['Ticker'])['Adj Close'].transform(lambda x: x.pct_change())

df.sort_index(inplace=True)
df.head()

Price,Date,Ticker,Adj Close,diffs
0,2024-01-01,A,130.100006,NaN
1,2024-01-01,AAL,14.230000,NaN
10,2024-01-01,ADM,55.059299,NaN
100,2024-01-01,CMI,237.772491,NaN
1002,2024-03-01,A,146.220001,0.123905


In [ ]:
x = df.dropna(axis=0)
x.shape

(501, 4)

In [ ]:
x.Date.min()

Timestamp('2024-03-01 00:00:00')

In [ ]:
x.head()

Price,Date,Ticker,Adj Close,diffs
1002,2024-03-01,A,146.220001,0.123905
1003,2024-03-01,AAL,14.880000,0.045678
1004,2024-03-01,AAPL,169.119995,-0.081693
1005,2024-03-01,ABBV,181.059998,0.111861
1006,2024-03-01,ABNB,163.869995,0.136881


In [ ]:
x.diffs.describe()

count    501.000000
mean       0.050236
std        0.085255
min       -0.280329
25%       -0.002453
50%        0.045527
75%        0.094976
max        0.475246
Name: diffs, dtype: float64

In [ ]:
x.to_csv('price_changes.csv')